In [27]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [28]:
import torch_geometric.utils
import torch

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [31]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [32]:
import lightning.pytorch as pl
import PPIGraph

# Test custom torch dataset

In [33]:
from torch_geometric.loader import DataLoader


In [34]:
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}


In [35]:
graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset = PPIGraph.GraphDatasetMLP(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [36]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: GraphDatasetMLP(2064):
Number of graphs: 2064
Number of features: 5
Number of classes: 2


In [37]:
print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')

Train set: 992, test set: 825, val set: 247


In [38]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(x=[32, 5], y=[32], name=[32], batch=[32], ptr=[33])



# Lightning training

In [39]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [40]:
# condition = 'base'
# checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f'MLP_{condition}' 
# project_name = f'PLA_041623_{condition}'

In [41]:
# AVAIL_GPUS = [1]
# BATCH_SIZE = 64 if AVAIL_GPUS else 32

# # Setting the seed
# pl.seed_everything(42)

# num_layers = [2,3,4]
# hiddens = [8, 16, 32, 64]
# pool = 'mean'

# epochs = 30

In [42]:
# for NUM_LAYERS, HIDDEN_CHANNELS in list(itertools.product(*[num_layers, hiddens])):
    
#     # Path to the folder where the pretrained models are saved
#     CHECKPOINT_PATH = checkpoint_folder / f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_count_cyto'
#     CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
    
#     # Run training
#     models = ['MLP']
#     for model_name in models:
#         run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_count_cyto')
#         model, result, trainer = PPIGraph.train_graph_classifier(model_name, train_set, val_set, test_set, 
#                                                                  dataset, CHECKPOINT_PATH, AVAIL_GPUS, 
#                                                                  in_channels=HIDDEN_CHANNELS,
#                                                                  hidden_channels=HIDDEN_CHANNELS, 
#                                                                  out_channels=HIDDEN_CHANNELS, 
#                                                                  num_layers=NUM_LAYERS, 
#                                                                  epochs=epochs,
#                                                                  embedding=True,
#                                                                  graph_pooling=pool)
#         run.finish()

In [43]:
# for NUM_LAYERS, HIDDEN_CHANNELS in list(itertools.product(*[num_layers, hiddens])):
    
#     # Path to the folder where the pretrained models are saved
#     CHECKPOINT_PATH = checkpoint_folder / f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_count_cell'
#     CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
    
#     # Run training
#     models = ['MLP']
#     for model_name in models:
#         run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_count_cell')
#         model, result, trainer = PPIGraph.train_graph_classifier(model_name, train_set, val_set, test_set, 
#                                                                  dataset, CHECKPOINT_PATH, AVAIL_GPUS, 
#                                                                  in_channels=HIDDEN_CHANNELS,
#                                                                  hidden_channels=HIDDEN_CHANNELS, 
#                                                                  out_channels=HIDDEN_CHANNELS, 
#                                                                  num_layers=NUM_LAYERS, 
#                                                                  epochs=epochs,
#                                                                  embedding=True,
#                                                                  graph_pooling=pool)
#         run.finish()

# K Fold

In [44]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [45]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [46]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset_subcellular = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset_subcellular, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [47]:
len(dataset_filtered)

1522

In [48]:
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"MLP_count_{condition}" 
project_name = f'PLA_050823_{condition}'

In [49]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16

epochs = 50

Global seed set to 42


In [50]:
# K fold nuclei cyto seperated
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    
    # Path to the folder where the pretrained models are saved
    CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_cell_{fold}' 
    CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

    # Skip already trained kfold and pool
    checkpoint = CHECKPOINT_PATH / "GraphLevelMLP" / "GraphLevelMLP.ckpt" 
    if checkpoint.exists():
        print(checkpoint)
        continue

    # Run training
    run = wandb.init(project=project_name, name=f'MLP_{NUM_LAYERS}_{HIDDEN_CHANNELS}_cell_{fold}', 
                    group=f'MLP_cell')
    PPIGraph.train_graph_classifier_kfold('MLP', 
                                         train_subset, 
                                         val_subset, 
                                         dataset, 
                                         CHECKPOINT_PATH, 
                                         AVAIL_GPUS, 
                                         in_channels=5,
                                         hidden_channels=HIDDEN_CHANNELS, 
                                         out_channels = HIDDEN_CHANNELS,
                                         num_layers=NUM_LAYERS, 
                                         epochs=epochs,
                                         embedding=False,
                                         batch_size=128,
                                         graph_pooling='mean')
    run.finish()


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python3

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇█▇▇▇█▇▇██▇▇▇
train_auc,▂▁▂▁▃▂▂▂▂▂▃▃▃▃▄▅▅▅▆▅▅▆▆▆▇▆▆▇█▇▇▆█▇▆▇▇▇▇▇
train_f1,▇▆▆▅▄▁▁▁▁▁▂▃▃▃▄▅▅▅▆▅▅▇▆▆▇▇▇▇█▇▇▇█▇▆▇▇▇▇▇
train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▄▃▃▃▂▂▂▂▃▂▂▂▂▃▂▃▂▂▂▂▂▃▂▂▂▂▂▂▁▂▂▂▁▂▂▂▁▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▃▁▃▃▂▂▂▂▂▂▄▄▂▅▅▅▆▆▆▆▃▃▇▃▃▃▇▃▃██▃
val_acc,▂▁▂▂▂▂▂▂▂▂▂▂▂▃▄▅▆▆▆▅▆▇▇▆█▇█▇▇█▇█▇██▇████
val_auc,▁▁▁▁▁▁▁▁▁▁▂▁▁▃▄▄▅▆▆▄▆▇▇▆█▇████▇▇▇███████
val_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▅▆▆▇▅▆▇▇▇█▇██████████████


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=50` reache

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▂▅▅▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇█▇██▇█▇█▇▇█████
train_auc,▁▁▁▁▂▂▁▂▂▁▂▃▄▄▅▆▆▆▇▅▆▇▆▇▆█▇██▇▇▇█▇▆▇▇█▇▇
train_f1,▆▆▆▆▄▂▁▁▁▁▁▃▄▄▅▇▆▆▇▆▇▇▇▇▆█▇██▇▇▇█▇▆▇▇█▇▇
train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▅▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▂▂▃▂▂▂▂▂▂▁▁▂▂▁▂▁▁▁▂▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▁▃▃▂▂▂▂▂▂▄▄▂▅▅▅▆▆▆▆▃▃▇▃▃▃▇▃▃██▃
val_acc,▂▁▃▃▃▃▃▃▃▃▃▃▄▃▇▇▇██▇██▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇
val_auc,▁▁▂▂▂▂▂▂▂▂▂▂▃▃▆▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█▆▇▇█▇▇▇
val_f1,▁▂▁▁▁▁▁▁▁▁▁▁▃▂▆▇▇▇▇▇▇▇▇█▇█▇██▇▇▇█▇▇▇█▇▇█


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python3

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▃▅▄▅▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇██▇
train_auc,▁▁▂▁▂▁▁▁▂▃▂▂▃▃▃▄▄▄▅▅▄▅▅▆▅▆▆▅▆▆▇▆▇▆▆▆▆▇█▇
train_f1,▆▇▇▅▄▂▂▁▂▃▃▂▃▃▃▄▄▅▅▆▄▆▆▆▅▆▇▆▆▆▇▇▇▆▇▆▆▇█▇
train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▁▂▂▂▁▁▁▁▁▁▁▂▂▂▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▁▃▃▂▂▂▂▂▂▄▄▂▅▅▅▆▆▆▆▃▃▇▃▃▃▇▃▃██▃
val_acc,▁▁▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▅▄▅▇▆▆▅██▅▇▇▇█▇▇█▆████
val_auc,▁▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▅▃▄▇▅▅▄██▅▆▇▇█▇▇▇▆████
val_f1,▃▄▁▁▁▁▁▁▁▁▁▁▁▂▂▂▄▄▆▄▅█▆▆▅██▆▇█▇█▇██▆████


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=50` reache

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▅▄▅▅▅▆▅▅▆▅▆▆▇▇▆▆▇█▇▇▇▇█▇▇█████▇▇███▇
train_auc,▁▁▂▁▃▁▂▁▂▂▂▂▃▃▃▄▅▅▅▄▅▆▅▆▅▆▇▆▇▇▇▇▇▇▇▆▇▇█▆
train_f1,▇▇▇▅▄▂▂▁▂▂▂▃▄▃▄▅▅▅▅▄▆▆▆▆▅▆▇▆▇▇▇▇▇▇▇▇▇▇█▆
train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▂▂▂▂▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▁▃▃▂▂▂▂▂▂▄▄▂▅▅▅▆▆▆▆▃▃▇▃▃▃▇▃▃██▃
val_acc,▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▄▅▅▅▄▆▇▇▇▆▇▇▇▇▇▇█▇██▇▇▇▇▇
val_auc,▂▃▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▅▅▄▆▆▇▆▆▇▆▆▇▇▇█▆██▇▇█▇▇
val_f1,▄▅▁▁▁▁▁▁▁▁▁▁▂▂▃▄▅▅▅▄▆▇▇▇▆▇▇▇▇█▇█▇██▇▇█▇█


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=50` reache

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▃▃▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇▇██▇████████▇██
train_auc,▂▁▃▂▃▂▃▂▃▃▃▄▄▄▅▆▅▆▅▆▆▆█▇▆▇██▇▇▇▇██▇█▇▇▇█
train_f1,▇▆▇▅▄▁▁▁▂▂▃▃▄▄▄▆▅▆▆▆▆▆█▇▆▆██▇▇█▇██▇█▇▇▇█
train_loss_epoch,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▄▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▃▂▁▂▂▂▁▁▁▂▁▁▂▂▁▁▂▂▂▂▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▁▃▃▂▂▂▂▂▂▄▄▂▅▅▅▆▆▆▆▃▃▇▃▃▃▇▃▃██▃
val_acc,▁▃▁▁▁▁▁▁▁▁▁▁▂▃▃▄▇▅▆▅▆▇█▆▆▆█▇▇▇▆▆▇▇▇▇▆▆▆▇
val_auc,▁▃▁▁▁▁▁▁▁▁▁▁▂▂▂▄▆▅▅▅▅▇█▆▅▆█▇▇▇▆▆▇▇▇▇▆▆▇▇
val_f1,▂▄▁▁▁▁▁▁▁▁▁▁▂▃▃▄▆▅▆▅▆▇█▆▆▇█▇▇▇▇▇▇▇▇█▇▇▇█
